In [3]:
from model_RA.utils.dataloader_keras import genUnbalSequence
#import tensorflow as tf
#from model_RA.dataset_RA import Dataset
import glob
#from model.utils.audio_utils import (bg_mix_batch, ir_aug_batch, load_audio,
#                                     get_fns_seg_list, load_audio_multi_start)
from model.utils.audio_utils import load_audio_multi_start
import tensorflow as tf
import wave
import os
import sys
import yaml
import numpy as np
import faiss
import os

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

In [5]:
def load_memmap_data(source_dir, fname, append_extra_length=None, shape_only=False):
    """
    Load data and datashape from the file path.

    Parameters
    ----------
    source_dir : (str)
        Directory where the files are located.
    fname : (str)
        File name except extension.
    append_extra_length : None or (int)
        Length to append empty vector when loading memmap.
    shape_only : (bool), optional
        Return only shape. The default is False.

    Returns
    -------
    (data, data_shape)
    """
    path_shape = os.path.join(source_dir, fname + '_shape.npy')
    path_data = os.path.join(source_dir, fname + '.mm')
    data_shape = np.load(path_shape)
    if shape_only:
        return data_shape

    if append_extra_length:
        data_shape[0] += append_extra_length
        data = np.memmap(path_data, dtype='float32', mode='r+',
                         shape=(data_shape[0], data_shape[1]))
    else:
        data = np.memmap(path_data, dtype='float32', mode='r',
                         shape=(data_shape[0], data_shape[1]))
    return data, data_shape

In [6]:
#load audio
#features
#nnfp
#generate
#predict

Config

In [7]:
import os
import sys
import yaml
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

In [8]:
config = "default_RA"
cfg = load_config(config)
#checkpoint_name = "Checks"

cli: Configuration from ./config/default_RA.yaml


Audio query

In [9]:
# BSZ
tr_batch_sz = cfg['BSZ']['TR_BATCH_SZ']
tr_n_anchor = cfg['BSZ']['TR_N_ANCHOR']
val_batch_sz = cfg['BSZ']['VAL_BATCH_SZ']
val_n_anchor = cfg['BSZ']['VAL_N_ANCHOR']
ts_batch_sz = cfg['BSZ']['TS_BATCH_SZ']

# Model parameters
dur = cfg['MODEL']['DUR']
hop = cfg['MODEL']['HOP']
fs = cfg['MODEL']['FS']

# Time-domain augmentation parameter
tr_snr = cfg['TD_AUG']['TR_SNR']
ts_snr = cfg['TD_AUG']['TS_SNR']
val_snr = cfg['TD_AUG']['VAL_SNR']
tr_use_bg_aug = cfg['TD_AUG']['TR_BG_AUG']
ts_use_bg_aug = cfg['TD_AUG']['TS_BG_AUG']
val_use_bg_aug = cfg['TD_AUG']['VAL_BG_AUG']
tr_use_ir_aug = cfg['TD_AUG']['TR_IR_AUG']
ts_use_ir_aug = cfg['TD_AUG']['TS_IR_AUG']
val_use_ir_aug = cfg['TD_AUG']['VAL_IR_AUG']
tr_use_speech_aug = cfg['TD_AUG']['TR_SPEECH_AUG']
ts_use_speech_aug = cfg['TD_AUG']['TS_SPEECH_AUG']
val_use_speech_aug = cfg['TD_AUG']['VAL_SPEECH_AUG']

val_bg_fps = None
val_ir_fps = None
val_speech_fps = None

seg_mode ='all'

In [10]:
def get_seg_list(fns_list=[],
                     segment_mode='all',
                     fs=22050,
                     duration=1,
                     hop=None):
    """
    return: fns_event_seg_list
        
        [[filename, seg_idx, offset_min, offset_max], [ ... ] , ... [ ... ]]
        
        offset_min is 0 or negative integer
        offset_max is 0 or positive integer
        
    """
    if hop == None: hop = duration
    fns_event_seg_list = []

    #print(f"fns_list={fns_list}")

    for offset_idx, filename in enumerate(fns_list):
        # Get audio info
        #print(f"filename={filename}")
        n_frames_in_seg = fs * duration
        n_frames_in_hop = fs * hop  # 2019 09.05
        file_ext = filename[-3:]

        if file_ext == 'wav':
            pt_wav = wave.open(filename, 'r')
            _fs = pt_wav.getframerate()

            if fs != _fs:
                raise ValueError('Sample rate should be {} but got {}'.format(
                    str(fs), str(_fs)))

            n_frames = pt_wav.getnframes()
            #n_segs = n_frames // n_frames_in_seg
            if n_frames > n_frames_in_seg:
                n_segs = (n_frames - n_frames_in_seg +
                          n_frames_in_hop) // n_frames_in_hop
            else:
                n_segs = 1

            n_segs = int(n_segs)
            assert (n_segs > 0)
            residual_frames = np.max([
                0,
                n_frames - ((n_segs - 1) * n_frames_in_hop + n_frames_in_seg)
            ])
            pt_wav.close()
        else:
            raise NotImplementedError(file_ext)

        # 'all', 'random_oneshot', 'first'
        if segment_mode == 'all':
            for seg_idx in range(n_segs):
                offset_min, offset_max = int(-1 *
                                             n_frames_in_hop), n_frames_in_hop
                if seg_idx == 0:  # first seg
                    offset_min = 0
                if seg_idx == (n_segs - 1):  # last seg
                    offset_max = residual_frames

                fns_event_seg_list.append(
                    [filename, seg_idx, offset_min, offset_max])
        elif segment_mode == 'random_oneshot':
            seg_idx = np.random.randint(0, n_segs)
            offset_min, offset_max = n_frames_in_hop, n_frames_in_hop
            if seg_idx == 0:  # first seg
                offset_min = 0
            if seg_idx == (n_segs - 1):  # last seg
                offset_max = residual_frames
            fns_event_seg_list.append(
                [filename, seg_idx, offset_min, offset_max])
        elif segment_mode == 'first':
            seg_idx = 0
            offset_min, offset_max = 0, 0
            fns_event_seg_list.append(
                [filename, seg_idx, offset_min, offset_max])
        else:
            raise NotImplementedError(segment_mode)

    return fns_event_seg_list

In [11]:
"""audiopath = '/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/test-query-db-500-30s/db/000/000134.wav'
audio_list =[audiopath]

fns_event_seg_list = get_fns_seg_list(audio_list,
                                        seg_mode,
                                        fs,
                                        dur,
                                        hop=hop)"""

"audiopath = '/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/test-query-db-500-30s/db/000/000134.wav'\naudio_list =[audiopath]\n\nfns_event_seg_list = get_fns_seg_list(audio_list,\n                                        seg_mode,\n                                        fs,\n                                        dur,\n                                        hop=hop)"

In [12]:
offset_margin_hop_rate=0.4
offset_margin_frame = int(hop * offset_margin_hop_rate * fs)
random_offset_anchor=False
n_pos_per_anchor = round((120 - 60) / 60)
amp_mode='normal'

In [13]:
def batch_load(anchor_idx_list, fns_event_seg_list):
        Xa_batch = None
        
        for idx in anchor_idx_list:  # idx: index for one sample
            pos_start_sec_list = []
            # fns_event_seg_list = [[filename, seg_idx, offset_min, offset_max], [ ... ] , ... [ ... ]]
            offset_min, offset_max = fns_event_seg_list[idx][
                2], fns_event_seg_list[idx][3]
            
            anchor_offset_min = np.max([offset_min, -offset_margin_frame])
            anchor_offset_max = np.min([offset_max, offset_margin_frame])
            
            
            _anchor_offset_frame = 0
            anchor_start_sec = fns_event_seg_list[idx][1] * hop


            """ Calculate multiple(=self.n_pos_per_anchor) pos_start_sec. """
            if n_pos_per_anchor > 0:
                pos_offset_min = np.max([
                    (_anchor_offset_frame - offset_margin_frame),
                    offset_min
                ])
                pos_offset_max = np.min([
                    (_anchor_offset_frame + offset_margin_frame),
                    offset_max
                ])

                if pos_offset_min==pos_offset_max==0:
                    # Only the case of running extras/dataset2wav.py 
                    # as offset_margin_hot_rate=0
                    pos_start_sec_list = fns_event_seg_list[idx][
                        1] * hop
                    pos_start_sec_list = [pos_start_sec_list]
                    print('!!!!!!!!!!!!!!!!!!!!!!')
                    print(f"pos_start_sec_list:{pos_start_sec_list}")
                    print(f"[anchor_start_sec]:{[anchor_start_sec]}")

                else:
                    # Otherwise, we apply random offset to replicas 
                    _pos_offset_frame_list = np.random.randint(
                        low=pos_offset_min,
                        high=pos_offset_max,
                        size=n_pos_per_anchor)
                    _pos_offset_sec_list = _pos_offset_frame_list / fs
                    pos_start_sec_list = fns_event_seg_list[idx][
                        1] * hop + _pos_offset_sec_list  
            
            """
            load audio returns: [anchor, pos1, pos2,..pos_n]
            """
            #print(self.fns_event_seg_list[idx])
            start_sec_list = np.concatenate(
                ([anchor_start_sec], pos_start_sec_list))
            
            xs = load_audio_multi_start(fns_event_seg_list[idx][0],
                                        start_sec_list, dur, fs,
                                        amp_mode)  # xs: ((1+n_pos)),T)

            if Xa_batch is None:
                Xa_batch = xs[0, :].reshape((1, -1))
            else:
                Xa_batch = np.vstack((Xa_batch, xs[0, :].reshape(
                    (1, -1))))  # Xa_batch: (n_anchor, T)
                
        return Xa_batch

In [14]:
def getitem(idx, index_event, n_anchor, fns_event_seg_list):
    """ Get anchor (original) and positive (replica) samples. """
    #print(f"idx:{idx}")
    index_anchor_for_batch = index_event[idx *
                                n_anchor:(idx + 1) *
                                n_anchor]
    #print(f"index_anchor_for_batch:{index_anchor_for_batch}")
    
    if len(index_anchor_for_batch) == 0:
        return None
     
    Xa_batch = batch_load(index_anchor_for_batch, fns_event_seg_list)
    
    #print(f"Xa_batch:{Xa_batch}")

    global bg_sel_indices, speech_sel_indices

    Xa_batch = np.expand_dims(Xa_batch,
                                1).astype(np.float32)  # (n_anchor, 1, T)
    #print(f"Xa_batch_expandido:{Xa_batch}")
    
    return Xa_batch

In [15]:
audiopath = '/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/test-query-db-500-30s/db/000/000134.wav'
audio_list = [audiopath]

In [16]:
seg_list = get_seg_list(fns_list=audio_list,
                     segment_mode='all',
                     fs=8000,
                     duration=1.,
                     hop=.5)

n_samples = len(seg_list)
index_event = np.arange(n_samples)
n_anchor=60

In [17]:
n_iter = len(index_event) // n_anchor + (1 if len(index_event) % n_anchor != 0 else 0)#len(seg_list)
#print(f"index_event:{index_event}")

for i in range(n_iter):
    #print(i)
    X = getitem(i, index_event, n_anchor, seg_list)

    if X is None:
        pass

In [18]:
from model_RA.fp_RA.melspec.melspectrogram_RA import get_melspec_layer
from model_RA.fp_RA.nnfp import get_fingerprinter

m_pre = get_melspec_layer(cfg, trainable=False)
m_fp = get_fingerprinter(cfg, trainable=False)

In [19]:
X = tf.concat(X, axis=0)
feat = m_pre(X)  # (nA+nP, F, T, 1)
m_fp.trainable = False
emb_f = m_fp.front_conv(feat)  # (BSZ, Dim)

emb_gf = m_fp.div_enc(emb_f)
emb_gf = tf.math.l2_normalize(emb_gf, axis=1)

In [20]:
emb_gf

<tf.Tensor: shape=(59, 128), dtype=float32, numpy=
array([[-0.02876278,  0.0361596 , -0.04542529, ..., -0.05875982,
         0.04338404,  0.16598956],
       [-0.05113657,  0.04920626, -0.02506815, ..., -0.04597016,
         0.02481091,  0.13902223],
       [-0.01189004, -0.00121552, -0.03751804, ...,  0.02125106,
         0.02168985,  0.17631531],
       ...,
       [ 0.02640655,  0.03909785, -0.0130725 , ..., -0.03039054,
         0.07897712,  0.18853524],
       [-0.0010752 ,  0.05746509, -0.00945702, ..., -0.05243146,
         0.00590391,  0.14370023],
       [-0.01910625,  0.02702098, -0.03873452, ..., -0.05920405,
        -0.03557371,  0.11459503]], dtype=float32)>

In [23]:
dim = cfg['MODEL']['EMB_SZ']
n_items = len(X) #ds[key].n_samples
arr_shape = (n_items, dim)

key = 'query-134'

output_root_dir = cfg['DIR']['OUTPUT_ROOT_DIR'] + \
            f'/{key}/'

os.makedirs(output_root_dir, exist_ok=True)

print(dim, n_items, arr_shape, output_root_dir)

128 59 (59, 128) ./logs/emb//query-134/


In [24]:
arr = np.memmap(f'{output_root_dir}/{key}.mm',
                dtype='float32',
                mode='w+',
                shape=arr_shape)
np.save(f'{output_root_dir}/{key}_shape.npy', arr_shape)

In [25]:
i=0
bsz = 125
arr[i * bsz:(i + 1) * bsz, :] = emb_gf.numpy()

In [26]:
arr

memmap([[-0.02876278,  0.0361596 , -0.04542529, ..., -0.05875982,
          0.04338404,  0.16598956],
        [-0.05113657,  0.04920626, -0.02506815, ..., -0.04597016,
          0.02481091,  0.13902223],
        [-0.01189004, -0.00121552, -0.03751804, ...,  0.02125106,
          0.02168985,  0.17631531],
        ...,
        [ 0.02640655,  0.03909785, -0.0130725 , ..., -0.03039054,
          0.07897712,  0.18853524],
        [-0.0010752 ,  0.05746509, -0.00945702, ..., -0.05243146,
          0.00590391,  0.14370023],
        [-0.01910625,  0.02702098, -0.03873452, ..., -0.05920405,
         -0.03557371,  0.11459503]], dtype=float32)

In [28]:
import os
import sys
import time
import glob
import click
import curses
import numpy as np
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(__file__))))
from eval_RA.utils.get_index_faiss import get_index
from eval_RA.utils.print_table import PrintTable


def load_memmap_data(source_dir,
                     fname,
                     append_extra_length=None,
                     shape_only=False,
                     display=True):
    path_shape = source_dir + fname + '_shape.npy'
    path_data = source_dir + fname + '.mm'
    data_shape = np.load(path_shape)
    if shape_only:
        return data_shape

    if append_extra_length:
        data_shape[0] += append_extra_length
        data = np.memmap(path_data, dtype='float32', mode='r+',
                         shape=(data_shape[0], data_shape[1]))
    else:
        data = np.memmap(path_data, dtype='float32', mode='r',
                         shape=(data_shape[0], data_shape[1]))
    if display:
        print(f'Load {data_shape[0]:,} items from \033[32m{path_data}\033[0m.')
    return data, data_shape

NameError: name '__file__' is not defined

In [ ]:
dim = cfg['MODEL']['EMB_SZ']
n_items = len(X) #ds[key].n_samples
arr_shape = (n_items, dim)
print(arr_shape)
key = 'query-134'

output_root_dir = cfg['DIR']['OUTPUT_ROOT_DIR'] + \
            f'/{key}/'


arr = np.memmap(f'{output_root_dir}/{key}.mm',
                dtype='float32',
                mode='w+',
                shape=arr_shape)
np.save(f'{output_root_dir}/{key}_shape.npy', arr_shape)

i=0
arr[i * bsz:(i + 1) * bsz, :] = emb.numpy()

In [ ]:
# Generate
sz_check = dict() # for warning message
for key in ds.keys():
    bsz = int(cfg['BSZ']['TS_BATCH_SZ'])  # Do not use ds.bsz here.
    # n_items = len(ds[key]) * bsz
    n_items = ds[key].n_samples
    dim = cfg['MODEL']['EMB_SZ']
    """
    Why use "memmap"?

    • First, we need to store a huge uncompressed embedding vectors until
        constructing a compressed DB with IVF-PQ (using FAISS). Handling a
        huge ndarray is not a memory-safe way: "memmap" consume 0 memory.

    • Second, Faiss-GPU does not support reconstruction of DB from
        compressed DB (index). In eval/eval_faiss.py, we need uncompressed
        vectors to calaulate sequence-level matching score. The created
        "memmap" will be reused at that point.

    Reference:
        https://numpy.org/doc/stable/reference/generated/numpy.memmap.html

    """
    # Create memmap, and save shapes
    assert n_items > 0
    arr_shape = (n_items, dim)
    arr = np.memmap(f'{output_root_dir}/{key}.mm',
                    dtype='float32',
                    mode='w+',
                    shape=arr_shape)
    np.save(f'{output_root_dir}/{key}_shape.npy', arr_shape)

    # Fingerprinting loop
    tf.print(
        f"=== Generating fingerprint from \x1b[1;32m'{key}'\x1b[0m " +
        f"bsz={bsz}, {n_items} items, d={dim}"+ " ===")
    progbar = Progbar(len(ds[key]))

    """ Parallelism to speed up preprocessing------------------------- """
    enq = tf.keras.utils.OrderedEnqueuer(ds[key],
                                            use_multiprocessing=True,
                                            shuffle=False)
    enq.start(workers=cfg['DEVICE']['CPU_N_WORKERS'],
                max_queue_size=cfg['DEVICE']['CPU_MAX_QUEUE'])
    i = 0
    while i < len(enq.sequence):
        progbar.update(i)
        X, _ = next(enq.get())
        emb = test_step(X, m_pre, m_fp)
        arr[i * bsz:(i + 1) * bsz, :] = emb.numpy() # Writing on disk.
        i += 1
    progbar.update(i, finalize=True)
    enq.stop()
    """ End of Parallelism-------------------------------------------- """

    tf.print(f'=== Succesfully stored {arr_shape[0]} fingerprint to {output_root_dir} ===')
    sz_check[key] = len(arr)
    arr.flush(); del(arr) # Close memmap

In [ ]:
n_iter = len(seg_list)

for i in range(n_iter):
    X = ds.__getitem__(i)
        
getitem(idx, index_event, n_anchor, seg_list)

In [ ]:
source_root_dir = cfg['DIR']['SOURCE_ROOT_DIR']
source_fps = sorted(
            glob.glob(source_root_dir + 'val-query-db-500-30s/' +
                      '**/*.wav', recursive=True))[:250]
       
ds_query = genUnbalSequence(
    source_fps,
    val_batch_sz,
    val_n_anchor,
    dur,
    hop,
    fs,
    shuffle=False,
    random_offset_anchor=False,
    bg_mix_parameter=[val_use_bg_aug, val_bg_fps, val_snr],
    ir_mix_parameter=[val_use_ir_aug, val_ir_fps],
    speech_mix_parameter=[val_use_speech_aug, val_speech_fps,
                            val_snr])

In [15]:
fns_event_seg_list = get_fns_seg_list(audio_list,
                                        seg_mode,
                                        fs,
                                        dur,
                                        hop=hop)

In [23]:
len(fns_event_seg_list)

59

In [22]:
fns_event_seg_list[3][1] * dur

3.0

In [ ]:
idx=0
for idx in fns_event_seg_list:
    print(fns_event_seg_list[idx])

In [ ]:
filename=str()
seg_start_sec=0.0#float()
offset_sec=0.0,
seg_length_sec=float()
seg_pad_offset_sec=0.0
fs=8000
amp_mode='normal'

for offset_idx, filename_audio in enumerate(audio_list):
    filename = filename_audio

#print(filename)
for idx in len(fns_event_seg_list):
    
fns_event_seg_list[idx][1] * dur
start_frame_idx = np.floor((seg_start_sec + offset_sec) * fs).astype(int)
seg_length_frame = np.floor(seg_length_sec * fs).astype(int)
end_frame_idx = start_frame_idx + seg_length_frame

print(start_frame_idx, seg_length_frame, end_frame_idx)

In [ ]:
filename=str()
seg_start_sec=float()
offset_sec=0.0,
seg_length_sec=float()
seg_pad_offset_sec=0.0
fs=8000
amp_mode='normal'

for offset_idx, filename_audio in enumerate(audio_list):
    filename = filename_audio

#print(filename)
start_frame_idx = np.floor((seg_start_sec + offset_sec) * fs).astype(int)
seg_length_frame = np.floor(seg_length_sec * fs).astype(int)
end_frame_idx = start_frame_idx + seg_length_frame

print(start_frame_idx, seg_length_frame, end_frame_idx)

# Get file-info
file_ext = filename[-3:]
#print(start_frame_idx, end_frame_idx)

if file_ext == 'wav':
    pt_wav = wave.open(filename, 'r')
    pt_wav.setpos(start_frame_idx)
    x = pt_wav.readframes(end_frame_idx - start_frame_idx)
    x = np.frombuffer(x, dtype=np.int16)
    x = x / 2**15  # dtype=float
else:
    raise NotImplementedError(file_ext)

# Max Normalize, random amplitude
if amp_mode == 'normal':
    pass
elif amp_mode == 'max_normalize':
    _x_max = np.max(np.abs(x))
    if _x_max != 0:
        x = x / _x_max
else:
    raise ValueError('amp_mode={}'.format(amp_mode))

# padding process. it works only when win_size> audio_size and padding='random'
audio_arr = np.zeros(int(seg_length_sec * fs))
seg_pad_offset_idx = int(seg_pad_offset_sec * fs)
audio_arr[seg_pad_offset_idx:seg_pad_offset_idx + len(x)] = x
   

In [12]:
audioDir = '/mnt/dataset/public/Fingerprinting/query_procura'
source_fps = sorted(
            glob.glob(audioDir +
                      '/*.wav', recursive=True))

ds_query = genUnbalSequence(
    source_fps,
    val_batch_sz,
    val_n_anchor,
    dur,
    hop,
    fs,
    shuffle=False,
    random_offset_anchor=False,
    bg_mix_parameter=[val_use_bg_aug, val_bg_fps, val_snr],
    ir_mix_parameter=[val_use_ir_aug, val_ir_fps],
    speech_mix_parameter=[val_use_speech_aug, val_speech_fps,
                            val_snr])

['/mnt/dataset/public/Fingerprinting/query_procura/000134.wav']


TypeError: 'NoneType' object is not iterable

In [31]:
source_fps = sorted(
            glob.glob(audiopath, recursive=True))

In [32]:
fns_event_seg_list = get_fns_seg_list(source_fps,
                                        seg_mode,
                                        fs,
                                        dur,
                                        hop=hop)

In [33]:
n_iter = len(fns_event_seg_list)
for i in range(n_iter):
        X = fns_event_seg_list.__getitem__(i)

In [34]:
dataset=Dataset(cfg)
ds = dataset.get_val_ds(max_song=2) # max 500

['/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/val-query-db-500-30s/db/000/000458.wav', '/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/val-query-db-500-30s/db/000/000736.wav']
self.n_samples, self.index_event: (60, array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59]))


In [37]:
ds.ir_mix

True

In [ ]:
ds_query

In [13]:
@tf.function
def test_step(X, m_pre, m_fp):
    
    X=X[0]
    feat = m_pre(X)  # (nA, F, T, 1)

    m_fp.trainable = False

    emb_f = m_fp.front_conv(feat)  # (BSZ, Dim)
    emb_gf = m_fp.div_enc(emb_f)
    emb_gf = tf.math.l2_normalize(emb_gf, axis=1)
    return emb_gf # f(.), L2(f(.)), L2(g(f(.))

Features

In [14]:
from model_RA.fp_RA.melspec.melspectrogram_RA import get_melspec_layer
from model_RA.fp_RA.nnfp import get_fingerprinter

In [15]:
m_pre = get_melspec_layer(cfg, trainable=False)
m_fp = get_fingerprinter(cfg, trainable=False)

In [16]:
max_n_samples=3000
bsz=120
n_iter = min(len(ds), max_n_samples // bsz)
print(f"n_iter={n_iter}")
emb=dict()
for i in range(n_iter):
        X = ds.__getitem__(i)
        emb['g(f)'] = test_step(X, m_pre, m_fp)

n_iter=1
X_depois:Tensor("X:0", shape=(60, 1, 8000), dtype=float32)


In [ ]:
emb

In [ ]:
X = tf.concat(X, axis=0)
feat = m_pre(X)  # (nA+nP, F, T, 1)
m_fp.trainable = False
emb_f = m_fp.front_conv(feat)  # (BSZ, Dim)

In [5]:
def extract_features(audio_path):
    """
    Extract features from an audio file using librosa.

    Parameters
    ----------
    audio_path : (str)
        Path to the audio file.

    Returns
    -------
    features : (np.ndarray)
        Extracted feature vector.
    """
    y, sr = librosa.load(audio_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)
    return mfcc_mean

In [6]:
from eval_RA.utils.get_index_faiss import get_index

ModuleNotFoundError: No module named 'eval_RA'

In [4]:
def predict(emb_dir, query_vector, index_type='ivfpq', use_gpu=True, max_train=1e7, k=1):
    """
    Search for the closest match to the query vector in the database.

    Parameters
    ----------
    emb_dir : (str)
        Directory where the {db, dummy_db}.mm files are located.
    query_vector : (np.ndarray)
        Query vector to search for.
    index_type : (str)
        Type of FAISS index.
    use_gpu : (bool)
        Whether to use GPU.
    max_train : (int)
        Maximum number of items for index training.
    k : (int)
        Number of top results to return.

    Returns
    -------
    indices : (np.ndarray)
        Indices of the closest matches in the database.
    distances : (np.ndarray)
        Distances to the closest matches.
    """
    # Load database
    db, db_shape = load_memmap_data(emb_dir, 'db')

    # Create and train FAISS index
    index = get_index(index_type, db, db_shape, use_gpu, max_train)

    # Add items to index
    index.add(db)

    # Search for the query vector
    distances, indices = index.search(query_vector, k)
    return indices, distances

if __name__ == "__main__":
    emb_dir = '/mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/CHECK_BFTRI_100/101'  # embeddings directory
    audio_path = '/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/test-query-db-500-30s/db/000/000134.wav'  # query audio file

    # Extract features from the query audio
    query_vector = extract_features(audio_path).reshape(1, -1)  # Reshape to match expected input shape

    # Perform the search
    indices, distances = predict(emb_dir, query_vector)
    print(f"Indices: {indices}")
    print(f"Distances: {distances}")

TypeError: Wrong number or type of arguments for overloaded function 'new_IndexFlatL2'.
  Possible C/C++ prototypes are:
    faiss::IndexFlatL2::IndexFlatL2(faiss::Index::idx_t)
    faiss::IndexFlatL2::IndexFlatL2()


In [1]:
# -*- coding: utf-8 -*-
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.
""" generate.py """
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Progbar
from model_RA.dataset_RA import Dataset
from model_RA.fp_RA.melspec.melspectrogram_RA import get_melspec_layer
from model_RA.fp_RA.nnfp import get_fingerprinter


def build_fp(cfg):
    """ Build fingerprinter """
    # m_pre: log-power-Mel-spectrogram layer, S.
    m_pre = get_melspec_layer(cfg, trainable=False)

    # m_fp: fingerprinter g(f(.)).
    m_fp = get_fingerprinter(cfg, trainable=False)
    return m_pre, m_fp


def load_checkpoint(checkpoint_root_dir, checkpoint_name, checkpoint_index,
                    m_fp):
    """ Load a trained fingerprinter """
    # Create checkpoint
    checkpoint = tf.train.Checkpoint(model=m_fp)
    checkpoint_dir = checkpoint_root_dir + f'/{checkpoint_name}/'
    c_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir,
                                           max_to_keep=None)

    # Load
    if checkpoint_index == None:
        tf.print("\x1b[1;32mArgument 'checkpoint_index' was not specified.\x1b[0m")
        tf.print('\x1b[1;32mSearching for the latest checkpoint...\x1b[0m')
        latest_checkpoint = c_manager.latest_checkpoint
        if latest_checkpoint:
            checkpoint_index = int(latest_checkpoint.split(sep='ckpt-')[-1])
            status = checkpoint.restore(latest_checkpoint)
            status.expect_partial()
            tf.print(f'---Restored from {c_manager.latest_checkpoint}---')
        else:
            raise FileNotFoundError(f'Cannot find checkpoint in {checkpoint_dir}')
    else:
        checkpoint_fpath = checkpoint_dir + 'ckpt-' + str(checkpoint_index)
        status = checkpoint.restore(checkpoint_fpath) # Let TF to handle error cases.
        status.expect_partial()
        tf.print(f'---Restored from {checkpoint_fpath}---')
    return checkpoint_index


def prevent_overwrite(key, target_path):
    if (key == 'dummy_db') & os.path.exists(target_path):
        answer = input(f'{target_path} exists. Will you overwrite (y/N)?')
        if answer.lower() not in ['y', 'yes']: sys.exit()


def get_data_source(cfg, source_root_dir, skip_dummy):
    dataset = Dataset(cfg)
    ds = dict()
    if source_root_dir:
        ds['custom_source'] = dataset.get_custom_db_ds(source_root_dir)
    else:
        if skip_dummy:
            tf.print("Excluding \033[33m'dummy_db'\033[0m from source.")
            pass
        else:
            ds['dummy_db'] = dataset.get_test_dummy_db_ds()

        if dataset.datasel_test_query_db in ['unseen_icassp', 'unseen_syn']:
            ds['query'], ds['db'] = dataset.get_test_query_db_ds()
        else:
            raise ValueError(dataset.datasel_test_query_db)

    tf.print(f'\x1b[1;32mData source: {ds.keys()}\x1b[0m',
             f'{dataset.datasel_test_query_db}')
    return ds


@tf.function
def test_step(X, m_pre, m_fp):
    """ Test step used for generating fingerprint """
    # X is not (Xa, Xp) here. The second element is reduced now.
    m_fp.trainable = False
    return m_fp(m_pre(X))  # (BSZ, Dim)

2024-06-07 16:15:50.364946: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
dim = cfg['MODEL']['EMB_SZ']

assert n_items > 0
arr_shape = (n_items, dim)
arr = np.memmap(f'{output_root_dir}/{key}.mm',
                dtype='float32',
                mode='w+',
                shape=arr_shape)
np.save(f'{output_root_dir}/{key}_shape.npy', arr_shape)

arr[i * bsz:(i + 1) * bsz, :] = emb.numpy() # Writing on disk.
i += 1


sz_check[key] = len(arr)
arr.flush(); del(arr) # Close memmap

In [ ]:
dim = cfg['MODEL']['EMB_SZ']
n_items = len(X) #ds[key].n_samples
arr_shape = (n_items, dim)
print(arr_shape)
key = 'query-134'

output_root_dir = cfg['DIR']['OUTPUT_ROOT_DIR'] + \
            f'/{key}/'


arr = np.memmap(f'{output_root_dir}/{key}.mm',
                dtype='float32',
                mode='w+',
                shape=arr_shape)
np.save(f'{output_root_dir}/{key}_shape.npy', arr_shape)

i=0
arr[i * bsz:(i + 1) * bsz, :] = emb.numpy()

In [ ]:
def generate_fingerprint(cfg,
                         checkpoint_name,
                         checkpoint_index,
                         source_root_dir,
                         output_root_dir,
                         skip_dummy):
    """
    After run, the output (generated fingerprints) directory will be:
      .
      └──logs
         └── emb
             └── CHECKPOINT_NAME
                 └── CHECKPOINT_INDEX
                     ├── db.mm
                     ├── db_shape.npy
                     ├── dummy_db.mm
                     ├── dummy_db_shape.npy
                     ├── query.mm
                     └── query_shape.npy
    """
    # Build and load checkpoint
    m_pre, m_fp = build_fp(cfg)
    checkpoint_root_dir = cfg['DIR']['LOG_ROOT_DIR'] + 'checkpoint/'
    checkpoint_index = load_checkpoint(checkpoint_root_dir, checkpoint_name,
                                       checkpoint_index, m_fp)

    # Get data source
    """ ds = {'key1': <Dataset>, 'key2': <Dataset>, ...} """
    ds = get_data_source(cfg, source_root_dir, skip_dummy)

    # Make output directory
    if output_root_dir:
        output_root_dir = output_root_dir + f'/{checkpoint_name}/{checkpoint_index}/'
    else:
        output_root_dir = cfg['DIR']['OUTPUT_ROOT_DIR'] + \
            f'/{checkpoint_name}/{checkpoint_index}/'
    os.makedirs(output_root_dir, exist_ok=True)
    if not skip_dummy:
        prevent_overwrite('dummy_db', f'{output_root_dir}/dummy_db.mm')

    # Generate
    sz_check = dict() # for warning message
    for key in ds.keys():
        bsz = int(cfg['BSZ']['TS_BATCH_SZ'])  # Do not use ds.bsz here.
        # n_items = len(ds[key]) * bsz
        n_items = ds[key].n_samples
        dim = cfg['MODEL']['EMB_SZ']
        """
        Why use "memmap"?

        • First, we need to store a huge uncompressed embedding vectors until
          constructing a compressed DB with IVF-PQ (using FAISS). Handling a
          huge ndarray is not a memory-safe way: "memmap" consume 0 memory.

        • Second, Faiss-GPU does not support reconstruction of DB from
          compressed DB (index). In eval/eval_faiss.py, we need uncompressed
          vectors to calaulate sequence-level matching score. The created
          "memmap" will be reused at that point.

        Reference:
            https://numpy.org/doc/stable/reference/generated/numpy.memmap.html

        """
        # Create memmap, and save shapes
        assert n_items > 0
        arr_shape = (n_items, dim)
        arr = np.memmap(f'{output_root_dir}/{key}.mm',
                        dtype='float32',
                        mode='w+',
                        shape=arr_shape)
        np.save(f'{output_root_dir}/{key}_shape.npy', arr_shape)

        # Fingerprinting loop
        tf.print(
            f"=== Generating fingerprint from \x1b[1;32m'{key}'\x1b[0m " +
            f"bsz={bsz}, {n_items} items, d={dim}"+ " ===")
        progbar = Progbar(len(ds[key]))

        """ Parallelism to speed up preprocessing------------------------- """
        enq = tf.keras.utils.OrderedEnqueuer(ds[key],
                                              use_multiprocessing=True,
                                              shuffle=False)
        enq.start(workers=cfg['DEVICE']['CPU_N_WORKERS'],
                  max_queue_size=cfg['DEVICE']['CPU_MAX_QUEUE'])
        i = 0
        while i < len(enq.sequence):
            progbar.update(i)
            X, _ = next(enq.get())
            emb = test_step(X, m_pre, m_fp)
            arr[i * bsz:(i + 1) * bsz, :] = emb.numpy() # Writing on disk.
            i += 1
        progbar.update(i, finalize=True)
        enq.stop()
        """ End of Parallelism-------------------------------------------- """

        tf.print(f'=== Succesfully stored {arr_shape[0]} fingerprint to {output_root_dir} ===')
        sz_check[key] = len(arr)
        arr.flush(); del(arr) # Close memmap

    if 'custom_source' in ds.keys():
        pass;
    elif sz_check['db'] != sz_check['query']:
        print("\033[93mWarning: 'db' and 'query' size does not match. This can cause a problem in evaluataion stage.\033[0m")
    return

In [39]:
import os
import yaml
import tensorflow as tf
import librosa
import numpy as np
import pickle
from model_RA.fp_RA.melspec.melspectrogram_RA import get_melspec_layer
from model_RA.fp_RA.nnfp import get_fingerprinter

# Configurações de ambiente
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Função para limpar a GPU
"""def clean_gpu():
    from numba import cuda 
    device = cuda.get_current_device()
    device.reset()
clean_gpu()

# Configuração da GPU no TensorFlow
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    print(e)"""



# Funções principais (assumindo que essas funções já estão definidas anteriormente)
def build_fp(cfg):
    m_pre = get_melspec_layer(cfg, trainable=False)

    m_fp = get_fingerprinter(cfg, trainable=False)

    return m_pre, m_fp

def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')
    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg



@tf.function
def predict(X, m_fp):
    emb_gf = m_fp(X)
    return emb_gf

def load_model():
    checkpoint_name_dir = "./logs/CHECK_BFTRI_100"
    config = "default_RA"
    cfg = load_config(config)
    _, m_fp = build_fp(cfg)
    checkpoint = tf.train.Checkpoint(m_fp)
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_name_dir))
    return m_fp

def run(filepath, fs, m_fp):
    if isinstance(filepath, str):
        print(f"file entrada: {filepath}")
        signal, fs = librosa.load(filepath, mono=True, sr=8000)

    if fs != 8000:
        signal = librosa.resample(signal, fs, 8000)
        fs = 8000

    fs = 8000
    win_sz = fs
    hop_sz = int(fs / 2)
    if len(signal) < win_sz:
        signal = librosa.util.pad_center(signal, size=win_sz, mode='constant')
    if len(signal) > 1.5 * win_sz:
        frames = np.transpose(librosa.util.frame(signal, frame_length=win_sz, hop_length=hop_sz))
    else:
        frames = signal[:fs][None, :]

    X = frames[np.newaxis, np.newaxis, ...]
    X = tf.convert_to_tensor(X, dtype=tf.float32)
    X = tf.transpose(X, perm=[2, 0, 1, 3])

    emb = predict(X, m_fp)
    tf.config.run_functions_eagerly(True)
    return emb.numpy()

# Função para carregar impressões digitais da base de dados
def load_database_fingerprints(database_path):
    with open(database_path, 'rb') as f:
        database_fingerprints = pickle.load(f)
    return database_fingerprints

def load_database_fingerprints_folder(database_folder):
    database_fingerprints = []
    
    # Percorrer todos os arquivos na pasta
    for filename in os.listdir(database_folder):
        file_path = os.path.join(database_folder, filename)
        
        # Verificar se o arquivo é um arquivo pickle
        if os.path.isfile(file_path) and file_path.endswith('.pkl'):
            with open(file_path, 'rb') as f:
                fingerprint = pickle.load(f)
                database_fingerprints.append(fingerprint)
    
    return database_fingerprints

# Função para calcular a similaridade entre dois vetores
def calculate_similarity(emb1, emb2):
    #return np.dot(emb1[0], emb2[0])
    return np.dot(emb1[0], emb2[0]) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))

# Função principal para prever se o query está na base de dados
def predict_query(query_file, database_fingerprints, similarity_threshold=0.8):
    # Carrega o modelo
    model_fp = load_model()
    
    # Carrega e processa o arquivo de áudio do query
    signal, sampling_rate = librosa.load(query_file, sr=None)
    
    # Gera a impressão digital do áudio do query
    emb_vector = run(query_file, sampling_rate, model_fp)
    
    # Verifica a similaridade com cada impressão digital na base de dados
    for db_emb in database_fingerprints:
        #print(f"db_emb:{db_emb.shape}")
        #print(f"emb_vector:{emb_vector.shape}")
        #print(f"dot:{np.dot(emb_vector[0], db_emb[0])}")
        #print(f"linalog:{np.linalg.norm(db_emb)}")
        print(f"similaridade:{np.dot(emb_vector[0].flatten(), db_emb[0].flatten()) / (np.linalg.norm(emb_vector[0].flatten()) * np.linalg.norm(db_emb[0].flatten()))}")
        similarity = calculate_similarity(emb_vector.flatten(), db_emb.flatten())
        if similarity >= similarity_threshold:
            print(f"Match found with similarity: {similarity}")
            return True
    
    print("No match found.")
    return False

# Exemplo de uso
database_path = '/mnt/dataset/public/Fingerprinting/features/fma_full/000'
#query_file = '/mnt/dataset/public/Fingerprinting/query_procura/000134.wav'
#query_file = '/mnt/dataset/public/Fingerprinting/query_procura/000003.wav'
query_file = '/mnt/dataset/public/Fingerprinting/query_procura/000020.wav'
#query_file = '/mnt/dataset/public/Fingerprinting/query_procura/000003.wav'
database_fingerprints = load_database_fingerprints_folder(database_path)
is_in_database = predict_query(query_file, database_fingerprints)

cli: Configuration from ./config/default_RA.yaml
file entrada: /mnt/dataset/public/Fingerprinting/query_procura/000020.wav
similaridade:0.14010010659694672
similaridade:0.15846888720989227
similaridade:-0.02840178646147251
similaridade:-0.14455148577690125
similaridade:-0.13392269611358643
similaridade:-0.13182249665260315
similaridade:-0.16113074123859406
similaridade:0.1485816389322281
similaridade:0.1693640947341919
similaridade:0.07374265044927597
similaridade:-0.1766311377286911
similaridade:-0.08311345428228378
similaridade:0.11754923313856125
similaridade:0.013537664897739887
similaridade:-0.15107092261314392
similaridade:0.0838080570101738
similaridade:-0.025540970265865326
similaridade:0.010827061720192432
similaridade:0.08970451354980469
similaridade:-0.0755641981959343
similaridade:-0.04691004008054733
similaridade:-0.02019057236611843
similaridade:0.020417416468262672
similaridade:-0.13800644874572754
similaridade:-0.05044283717870712
similaridade:0.08835076540708542
simila